In [125]:
import os
import pandas as pd
from IPython.display import display, HTML
import re

path = "c:\\data\\"

filename_read = os.path.join(path,"workload.csv")
df = pd.read_csv(filename_read)


# Experiment 1

In [81]:
import re

s = 'neural-r:sqrt-y0|rmse'
result = re.search('\:(.*)\|', s).group(1)
print(result.group(1))

sqrt-y0


In [146]:
lookup = {'diff-y0' : 'Difference',
'log-y0' : 'Log',
'poly-y0' : 'Polynomial',
'poly2-y0' : 'Polynomial2',
'pow-y0' : 'Power',
'ratio-y0' : 'Ratio',
'ratio_diff-y0' : 'Ratio Difference',
'ratio_poly-y0' : 'Ratio Polynomial',
'ratio_poly2-y0' : 'Ratio Polynomial2',
'sqrt-y0' : 'Square Root'}


ex1 = df[df.name == 'experiment-1'][['dataset','algorithm','predictors','cycle','result','result-raw']]
f = {'result':['min','mean','std']}
ex1_summary = ex1.groupby(['algorithm']).agg(f)

ex1_summary.columns = [x[1] for x in ex1_summary.columns.values]
ex1_summary.insert(0,'algorithm', ex1_summary.index)
ex1_summary.insert(1, 'name' ,[re.search('\:(.*)\|', x).group(1) for x in ex1_summary.algorithm])
ex1_summary.insert(0, 'set' , [('gp' if 'gp' in x else 'neural') for x in ex1_summary.algorithm])
ex1_summary = ex1_summary[ex1_summary.name.isin(lookup.keys())]
ex1_summary.insert(1, 'expression' ,[lookup[x] for x in ex1_summary.name])
ex1_summary.sort_values(by=['expression'],inplace=True)
ex1_summary.index = range(len(ex1_summary))
ex1_summary.insert(0, 'num', ["{}-{}".format(1,x+1) for x in range(len(ex1_summary))])



In [124]:
# Experiment 1 neural results
ex1_summary_neural = ex1_summary[ex1_summary['set'] == 'neural'][['num','expression','min','mean','std']]
display(ex1_summary_neural)
ex1_summary_neural.to_csv(os.path.join(path,"diss_ex1_neural.csv"),index=False)

,num,expression,min,mean,std
10,1-11,Difference,0.017476,0.654883,0.356321
11,1-12,Log,0.082320,0.387833,0.358690
12,1-13,Polynomial,0.151494,1.534651,1.796200
13,1-14,Polynomial2,0.374127,0.877179,0.313812
14,1-15,Power,0.238337,0.239450,0.000829
15,1-16,Ratio,27.453023,27.473170,0.011318
16,1-17,Ratio Difference,22.982326,22.982632,0.000684
17,1-18,Ratio Polynomial,10.195469,10.243054,0.026603
18,1-19,Ratio Polynomial2,0.059725,0.145600,0.078392
19,1-20,Square Root,0.046957,0.195941,0.083285


In [123]:
# Experiment 1 GP results
ex1_summary_gp = ex1_summary[ex1_summary['set'] == 'gp'][['num','expression','min','mean','std']]
display(ex1_summary_gp)
ex1_summary_gp.to_csv(os.path.join(path,"diss_ex1_gp.csv"),index=False)

,num,expression,min,mean,std
0,1-1,Difference,0.000000e+00,0.114616,0.256290
1,1-2,Log,5.708419e-01,0.796477,0.290220
2,1-3,Polynomial,1.129961e+00,1.286212,0.144640
3,1-4,Polynomial2,7.261021e-01,1.129642,0.228499
4,1-5,Power,0.000000e+00,0.064300,0.143779
5,1-6,Ratio,0.000000e+00,10.991299,15.050457
6,1-7,Ratio Difference,2.298330e+01,22.989943,0.010600
7,1-8,Ratio Polynomial,7.807116e+00,8.822704,1.334069
8,1-9,Ratio Polynomial2,1.287448e-01,0.262289,0.167348
9,1-10,Square Root,1.897332e-18,0.029122,0.039966


# Experiment 2

In [165]:
ex2 = df[df.name == 'experiment-2'][['dataset','algorithm','predictors','cycle','result','result-raw']]
#ex2 = ex2[ex2.dataset != 'feature_eng.csv']

f = {'result-raw':['min','mean','std']}
ex2_summary = ex2.groupby(['algorithm','dataset']).agg(f)

ex2_summary.columns = [x[1] for x in ex2_summary.columns.values]
ex2_summary.insert(0,'algorithm', [x[0] for x in ex2_summary.index])
ex2_summary.insert(0,'dataset', [x[1] for x in ex2_summary.index])
ex2_summary.insert(1, 'name' ,[re.search('(.*)\.', x).group(1) for x in ex2_summary.dataset])
ex2_summary.insert(0, 'set' , [('gp' if 'gp' in x else 'neural') for x in ex2_summary.algorithm])
ex2_summary.index = range(len(ex2_summary))
ex2_summary.sort_values(by=['set','name'],inplace=True)

ex2_summary_neural = ex2_summary[ex2_summary['set'] == 'neural'][['name','min','mean','std']]
ex2_summary_neural.columns = ['name','neural_min','neural_mean','neural_std']
ex2_summary_gp = ex2_summary[ex2_summary['set'] == 'gp'][['name','min','mean','std']]
ex2_summary_gp.columns = ['name','gp_min','gp_mean','gp_std']

#display(ex2_summary_neural)
#display(ex2_summary_gp)
display(pd.merge(ex2_summary_neural,ex2_summary_gp,how="left"),on=['name'])

,name,neural_min,neural_mean,neural_std,gp_min,gp_mean,gp_std
0,abalone,2.195472,2.270479,0.098477,1.141926,1.240801,0.062280
1,auto-mpg,2.295976,3.423209,2.314089,2.218571,3.958601,1.008322
2,bupa,0.407695,0.448088,0.030864,0.335325,0.357713,0.019246
3,covtype,0.356659,0.380583,0.017949,NaN,NaN,NaN
4,crx,0.328095,0.350344,0.019320,0.000000,0.000000,0.000000
5,forestfires,0.058114,0.061155,0.002623,NaN,NaN,NaN
6,glass,0.332655,0.346421,0.013476,NaN,NaN,NaN
7,heart_cleveland,0.313970,0.332433,0.019368,NaN,NaN,NaN
8,hepatitis,0.388333,0.436432,0.034477,0.000000,0.001345,0.002251
9,horse-colic,0.349074,0.368696,0.013137,NaN,NaN,NaN


# Experiment 3

In [180]:
ex3 = df[df.name == 'experiment-3'][['dataset','algorithm','predictors','cycle','result','result-raw']]

f = {'result-raw':['min','mean','std']}
ex3_summary = ex3.groupby(['dataset']).agg(f)

ex3_summary.columns = [x[1] for x in ex3_summary.columns.values]
ex3_summary.insert(0,'dataset', ex3_summary.index)
ex3_summary.insert(1, 'name' ,[re.search('(.*)\.', x).group(1) for x in ex3_summary.dataset])
ex3_summary.index = range(len(ex3_summary))
ex3_summary.drop('dataset', axis=1, inplace=True)
ex3_summary.columns = ['name','ensemble_min','ensemble_mean','ensemble_std']

ex3_summary = pd.merge(ex3_summary,ex2_summary_neural,how="left",on=['name'])
ex3_summary = pd.merge(ex3_summary,ex2_summary_gp,how="left",on=['name'])
ex3_summary.drop(['neural_std','neural_min','gp_std','gp_min'], axis=1, inplace=True)

display(ex3_summary)

,name,ensemble_min,ensemble_mean,ensemble_std,neural_mean,gp_mean
0,abalone,2.175957,2.425743,0.237759,2.270479,1.240801
1,auto-mpg,2.476902,5.542507,2.783188,3.423209,3.958601
2,bupa,0.438132,0.464637,0.023409,0.448088,0.357713
3,crx,0.312449,0.335433,0.017782,0.350344,0.000000
4,hepatitis,0.396677,0.447993,0.030636,0.436432,0.001345
5,pima-indians-diabetes,0.399727,0.427975,0.044008,0.438807,0.304308
6,wcbreast_wdbc,0.080322,0.106219,0.045468,0.089072,0.257292
7,wcbreast_wpbc,0.325037,0.386586,0.036454,0.385926,0.129152


# Experiment 6

In [214]:
ex6 = df[df.name == 'experiment-6'][['dataset','algorithm','predictors','cycle','result','result-raw']]

f = {'result-raw':['min','mean','std']}
ex6_summary = ex6.groupby(['algorithm','dataset']).agg(f)

ex6_summary.columns = [x[1] for x in ex6_summary.columns.values]
ex6_summary.insert(0,'algorithm', [x[0] for x in ex6_summary.index])
ex6_summary.insert(0,'dataset', [x[1] for x in ex6_summary.index])
ex6_summary.insert(1, 'name' ,[re.search('(.*)\.', x).group(1) for x in ex6_summary.dataset])
ex6_summary.index = range(len(ex6_summary))
ex6_summary.drop(['dataset','algorithm'], axis=1, inplace=True)
ex6_summary.columns = ['name','auto_min','auto_mean','auto_std']
ex6_summary = pd.merge(ex6_summary,ex3_summary,how="left",on=['name'])
ex6_summary = pd.merge(ex6_summary,ex2_summary_neural,how="left",on=['name'])
ex6_summary.neural_mean_x.fillna(ex6_summary.neural_mean_y,inplace=True)
ex6_summary.drop(['auto_min','auto_std','ensemble_min','ensemble_std','neural_mean_y','neural_min','neural_std'], 
                 axis=1, inplace=True)
ex6_summary.rename(columns={'neural_mean_x':'neural_mean'},inplace=True)
ex6_summary['best_non_fe'] = ex6_summary[['ensemble_mean','neural_mean','gp_mean']].min(axis=1,skipna=True)
ex6_summary['diff'] = ex6_summary.auto_mean - ex6_summary.best_non_fe
ex6_summary['pct'] = (ex6_summary.auto_mean - ex6_summary.best_non_fe) / ex6_summary.best_non_fe

ex6_summary.sort_values(by=['name'],inplace=True)
ex6_summary.index = range(len(ex6_summary))



display(ex6_summary)

,name,auto_mean,ensemble_mean,neural_mean,gp_mean,best_non_fe,diff,pct
0,abalone,2.650214,2.425743,2.270479,1.240801,1.240801,1.409413,1.135889
1,auto-mpg,8.764598,5.542507,3.423209,3.958601,3.423209,5.341389,1.560346
2,bupa,0.560429,0.464637,0.448088,0.357713,0.357713,0.202716,0.566700
3,covtype,0.360868,NaN,0.380583,NaN,0.380583,-0.019714,-0.051800
4,crx,0.336507,0.335433,0.350344,0.000000,0.000000,0.336507,inf
5,forestfires,0.060468,NaN,0.061155,NaN,0.061155,-0.000687,-0.011235
6,glass,0.387981,NaN,0.346421,NaN,0.346421,0.041560,0.119970
7,heart_cleveland,0.325934,NaN,0.332433,NaN,0.332433,-0.006499,-0.019550
8,hepatitis,0.454319,0.447993,0.436432,0.001345,0.001345,0.452974,336.693836
9,horse-colic,0.370963,NaN,0.368696,NaN,0.368696,0.002267,0.006148
